In [ ]:
import re

class WebApplicationFirewall:
    def __init__(self):
        self.sql_patterns = [r"(\bor\b.*=)", r"(;|--|#|%27|'|--|%23|%24|%2F)"]
        self.xss_patterns = [r"<script>", r"javascript:", r"onerror="]

    def is_sql_injection(self, query):
        for pattern in self.sql_patterns:
            if re.search(pattern, query, re.IGNORECASE):
                return True
        return False

    def is_xss_attack(self, input_text):
        for pattern in self.xss_patterns:
            if re.search(pattern, input_text, re.IGNORECASE):
                return True
        return False

    def inspect_request(self, request):
        url = request.get('url', '')
        query = request.get('query', '')
        headers = request.get('headers', {})
        body = request.get('body', '')

        if self.is_sql_injection(query) or self.is_sql_injection(body):
            print(f"Blocked SQL Injection Attempt on URL: {url}")
            return False
        if self.is_xss_attack(body) or self.is_xss_attack(url):
            print(f"Blocked XSS Attack Attempt on URL: {url}")
            return False
        return True

# Simulate incoming HTTP requests
requests = [
    {"url": "https://example.com/search", "query": "name=admin'--", "headers": {}, "body": ""},
    {"url": "https://example.com/login", "query": "", "headers": {}, "body": "<script>alert(1)</script>"},
    {"url": "https://example.com/page", "query": "id=10", "headers": {}, "body": ""}
]

# Instantiate WAF
waf = WebApplicationFirewall()
for request in requests:
    if waf.inspect_request(request):
        print(f"Request allowed: {request['url']}")
    else:
        print(f"Request blocked: {request['url']}")